We aim to show that using the ANTs function to apply a deformation field produces the same result as using an alternative function. First we import needed libraries and define functions

In [10]:
!pip install antspyx

# Libraries and functions
import nibabel as nib
import ants
import scipy.ndimage as sc
import numpy as np

def apply_with_ants(mri_path, warp_path, output_path,):
    mri = ants.image_read(mri_path)
    mri_disp = ants.apply_transforms(mri, mri, warp_path)
    ants.image_write(mri_disp, output_path)



def apply_warp_alternative(image, warp_field, interp_order, output_path_map):

    image = nib.load(image)
    affine = image.affine
    image = image.get_fdata()
    warp_field = nib.load(warp_field).get_fdata()

    # Extract the x, y, and z components of the warp field
    x_component = warp_field[:, :, :, 0,0]
    y_component = warp_field[:, :, :,0, 1]
    z_component = warp_field[:, :, :,0, 2]

    # Create an array of coordinates for the image and segmentation mask
    coordinates_image = np.indices(image.shape).astype(float)

    # Apply the warp field to the coordinates
    coordinates_image[0] += x_component
    coordinates_image[1] += y_component
    coordinates_image[2] += z_component

    # Use the map_coordinates function to apply the warp to the image
    deformed_image = sc.map_coordinates(
        image, coordinates_image, order=interp_order)

    deformed_nii = nib.Nifti1Image(deformed_image, affine)
    nib.save(deformed_nii, output_path_map)


def compute_ncc(arr1, arr2):
    # Check if the arrays have the same length
    if len(arr1) != len(arr2):
        raise ValueError("Arrays must have the same length")

    # Convert the arrays to numpy arrays to ensure calculations work properly
    arr1 = np.array(arr1)
    arr2 = np.array(arr2)

    # Compute the mean of each array
    mean_arr1 = np.mean(arr1)
    mean_arr2 = np.mean(arr2)

    # Compute the cross-correlation term
    cross_correlation = np.sum((arr1 - mean_arr1) * (arr2 - mean_arr2))

    # Compute the denominators for the normalization
    normalization_denominator1 = np.sqrt(np.sum((arr1 - mean_arr1) ** 2))
    normalization_denominator2 = np.sqrt(np.sum((arr2 - mean_arr2) ** 2))

    # Compute the normalized cross-correlation
    ncc = cross_correlation / (normalization_denominator1 * normalization_denominator2)

    return ncc

We have defined two functions: one that applies a displacement field using ANTs and another that applies the same displacement field using map_coordinates from the scipy.ndimage library. First, we apply the deformation with ANTs

In [11]:
mri_path = 'mri.nii.gz'
warp_field_path = 'warp.nii.gz'
output_path_ants = 'mri_disp_ants.nii.gz'

apply_with_ants(mri_path, warp_field_path, output_path_ants)


Then, we apply the same displacement field with map_coordinates

In [19]:
output_path_map = 'mri_disp_alternative.nii.gz'
apply_warp_alternative(mri_path, warp_field_path, interp_order = 3, output_path_map = output_path_map)

Finally, we compute the Cross-Correlation (NCC) between deformed image with ANTs and map_coordinates.

In [16]:
ants_image = nib.load(output_path_ants).get_fdata()
map_image = nib.load(output_path_map).get_fdata()

ncc = compute_ncc(ants_image, ants_image)

print(f'NCC value is {ncc}')

NCC value is 1.0


As we can see, the NCC value is 1, which confirms that applying a deformation field with either ANTs or map_coordinates yields the same result. This is because applying a deformation field is a deterministic process, independent of the method used